In [1]:
from attention import AttentionLayer

In [2]:
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
data=pd.read_csv("Reviews.csv",nrows=100000)

In [4]:
data.drop_duplicates(subset=['Text'],inplace=True)#dropping duplicates
data.dropna(axis=0,inplace=True)#dropping na

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88425 entries, 0 to 99999
Data columns (total 2 columns):
Summary    88425 non-null object
Text       88425 non-null object
dtypes: object(2)
memory usage: 2.0+ MB


In [6]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [7]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [8]:
#call the function
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [9]:
cleaned_text[:5]
print(type(cleaned_text))

<class 'list'>


In [10]:
#call the function
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [11]:
cleaned_summary[:5]

['good quality dog food',
 'not as advertised',
 'delight says it all',
 'cough medicine',
 'great taffy']

In [12]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
print(type(data['cleaned_text']))
(data['cleaned_text'].shape)

<class 'pandas.core.series.Series'>


(88425,)

# Understanding distribution of sequences

In [13]:
import matplotlib.pyplot as plt

text_word_count = []
summary_word_count = []

# populate the lists with sentence lengths
for i in data['cleaned_text']:
      text_word_count.append(len(i.split()))

for i in data['cleaned_summary']:
      summary_word_count.append(len(i.split()))

length_df = pd.DataFrame({'text':text_word_count, 'summary':summary_word_count})

length_df.hist(bins = 30)
plt.show()

<Figure size 640x480 with 2 Axes>

In [14]:
cnt=0
for i in data['cleaned_summary']:
    if(len(i.split())<=8):
        cnt=cnt+1
print(cnt/len(data['cleaned_summary']))

0.9425049476957874


In [15]:
cnt=0
for i in data['cleaned_text']:
    if(len(i.split())<=30):
        cnt=cnt+1
print(cnt/len(data['cleaned_text']))

0.5498105739327114


In [16]:
max_text_len=30
max_summary_len=8

In [17]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]
print(type(cleaned_text))
for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

<class 'numpy.ndarray'>


In [18]:
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [19]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=False)

# rare words and its coverage

In [20]:
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)
print(tot_cnt,cnt)

% of rare words in vocabulary: 66.02602844594867
Total Coverage of rare words: 2.941374689273152
24819 16387


# text tokenizer

In [21]:

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')


print(x_tr)

#size of vocabulary ( +1 for padding token)
x_voc = x_tokenizer.num_words + 1

print(x_voc)

[[  34  205 6128 ...    0    0    0]
 [   5  114 2195 ...    0    0    0]
 [ 139 2366  734 ...    0    0    0]
 ...
 [ 294  965 5293 ...    0    0    0]
 [   2  889   71 ...    0    0    0]
 [1472   14  349 ...    0    0    0]]
8433


# summary tokenizer

In [22]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

# rare wrods and its covererage

In [23]:
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)
print(tot_cnt,cnt)

% of rare words in vocabulary: 78.07734806629834
Total Coverage of rare words: 5.365097201932802
9050 7066


In [24]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print(y_voc)

1985


In [25]:
y_tokenizer.word_counts['sostok'],len(y_tr)

(42496, 42496)

# delete that contains only start and end

In [26]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [27]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

# model building

We are finally at the model building part. But before we do that, we need to familiarize ourselves with a few terms which are required prior to building the model.

Return Sequences = True: When the return sequences parameter is set to True, LSTM produces the hidden state and cell state for every timestep

Return State = True: When return state = True, LSTM produces the hidden state and cell state of the last timestep only

Initial State: This is used to initialize the internal states of the LSTM for the first timestep

Stacked LSTM: Stacked LSTM has multiple layers of LSTM stacked on top of each other. This leads to a better representation of the sequence. I encourage you to experiment with the multiple layers of the LSTM stacked on top of each other (it’s a great way to learn this)

Here, we are building a 3 stacked LSTM for the encoder:

In [28]:
from keras import backend as K 
from keras.models import load_model
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

W0810 21:33:09.340873  2956 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0810 21:33:09.341901  2956 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0810 21:33:09.356830  2956 deprecation_wrapper.py:119] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 21:33:09.359823  2956 deprecation.py:506] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\initializers.py:119: calli

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      843300      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 30, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [29]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [31]:
model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))
model.save_weights('trainedModel_weights3.h5')
model.load_weights('trainedModel_weights3.h5')

W0810 21:33:11.053363  2956 deprecation.py:323] From c:\users\user\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 41368 samples, validate on 4581 samples
Epoch 1/50
41368/41368 [==============================] - 82s 2ms/sample - loss: 2.8242 - acc: 0.5716 - val_loss: 2.5345 - val_acc: 0.5999
Epoch 2/50
41368/41368 [==============================] - 79s 2ms/sample - loss: 2.5084 - acc: 0.6019 - val_loss: 2.3549 - val_acc: 0.6147
Epoch 3/50
41368/41368 [==============================] - 79s 2ms/sample - loss: 2.3540 - acc: 0.6146 - val_loss: 2.2590 - val_acc: 0.6241
Epoch 4/50
41368/41368 [==============================] - 80s 2ms/sample - loss: 2.2569 - acc: 0.6225 - val_loss: 2.2070 - val_acc: 0.6308
Epoch 5/50
41368/41368 [==============================] - 82s 2ms/sample - loss: 2.1918 - acc: 0.6281 - val_loss: 2.1586 - val_acc: 0.6346
Epoch 6/50
41368/41368 [==============================] - 82s 2ms/sample - loss: 2.1390 - acc: 0.6327 - val_loss: 2.1334 - val_acc: 0.6376
Epoch 7/50
41368/41368 [==============================] - 81s 2ms/sample - loss: 2.0986 - acc: 0.6364 - val_loss: 2.1

# understanding the diagonastic plot

In [32]:
'''from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()'''

"from matplotlib import pyplot\npyplot.plot(history.history['loss'], label='train')\npyplot.plot(history.history['val_loss'], label='test')\npyplot.legend()\npyplot.show()"

In [33]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index
print(target_word_index)

{'sostok': 1, 'eostok': 2, 'great': 3, 'good': 4, 'the': 5, 'best': 6, 'not': 7, 'for': 8, 'love': 9, 'and': 10, 'coffee': 11, 'it': 12, 'my': 13, 'tea': 14, 'product': 15, 'delicious': 16, 'this': 17, 'is': 18, 'taste': 19, 'very': 20, 'but': 21, 'excellent': 22, 'yummy': 23, 'of': 24, 'to': 25, 'dog': 26, 'these': 27, 'flavor': 28, 'price': 29, 'tasty': 30, 'like': 31, 'favorite': 32, 'ever': 33, 'too': 34, 'in': 35, 'stuff': 36, 'loves': 37, 'food': 38, 'awesome': 39, 'snack': 40, 'so': 41, 'yum': 42, 'dogs': 43, 'are': 44, 'as': 45, 'just': 46, 'them': 47, 'chocolate': 48, 'healthy': 49, 'free': 50, 'nice': 51, 'on': 52, 'with': 53, 'perfect': 54, 'wonderful': 55, 'better': 56, 'you': 57, 'treats': 58, 'treat': 59, 'tastes': 60, 'no': 61, 'really': 62, 'tasting': 63, 'have': 64, 'than': 65, 'sweet': 66, 'what': 67, 'quality': 68, 'hot': 69, 'at': 70, 'cup': 71, 'buy': 72, 'bad': 73, 'popcorn': 74, 'chips': 75, 'amazing': 76, 'one': 77, 'little': 78, 'gluten': 79, 'cat': 80, 'mix': 

# inference

In [34]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [35]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [36]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [37]:
print(x_tr[0])
print(x_tr[0].shape)
print("Review:",seq2text(x_tr[0]))
print("Original summary:",seq2summary(y_tr[0]))
print("Predicted summary:",decode_sequence(x_tr[0].reshape(1,max_text_len)))
print("\n")

[  34  205 6128  550   30   25  143   43    2   80    5  473    3 2586
 1311  443  407   23 4117 1663 6129    5   23    0    0    0    0    0
    0    0]
(30,)
Review: bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better 
Original summary: good quality dog food 
Predicted summary:  my dog loves this




In [38]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      843300      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 30, 300), (N 481200      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [39]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.load_weights('trainedModel_weights3.h5')

In [40]:
for i in range(100):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better 
Original summary: good quality dog food 
Predicted summary:  my dog loves this


Review: product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended product jumbo 
Original summary: not as advertised 
Predicted summary:  melted


Review: looking secret ingredient believe found got addition root beer extract ordered made cherry soda flavor medicinal 
Original summary: cough medicine 
Predicted summary:  not the best


Review: great taffy great price wide assortment yummy taffy delivery quick taffy lover deal 
Original summary: great taffy 
Predicted summary:  great gift


Review: taffy great flavors soft chewy candy individually wrapped well none candies stuck together happen expensive version would highly recommend candy served beach themed party everyone loved 
Ori

Predicted summary:  not worth it


Review: tea flavor whole brunch flavors returnable wasted bucks 
Original summary: no tea flavor 
Predicted summary:  great tea


Review: taste great berries melted may order winter order cold weather enjoy flavor 
Original summary: order only in cold weather 
Predicted summary:  good but


Review: know cannot make tea good granted south know never enjoyed tea sweet without sweet tastes crisp 
Original summary: this is the best 
Predicted summary:  great tea


Review: peppermint stick delicious fun eat dad got one christmas remembered similar one little girl love 
Original summary: delicious 
Predicted summary:  great gift


Review: great gift ages purchased giant canes recipients loved much kept would eat 
Original summary: great 
Predicted summary:  great gift


Review: three dogs love food bought specifically one dogs food allergies works great hot spots tummy problems love ships right door free shipping 
Original summary: great food 
Predicted sum

Predicted summary:  best ever


Review: fabulous marinade love use chicken either baked oven grill enough flavor flair even used dinner parties receive rave reviews guests definitely worth price super cheap super easy love 
Original summary: great marinade 
Predicted summary:  great seasoning


Review: works chicken fish beef pork fast easy makes taste excellent plus buying bulk box stores 
Original summary: awesome stuff 
Predicted summary:  great seasoning


Review: got brother cruise diet decided try one actually tastes pretty good 
Original summary: tastes good 
Predicted summary:  good stuff


Review: singles sell store box singles sure amazon selling box singles hazelnut coffee creamer favorite truly good buy 
Original summary: rip off price 
Predicted summary:  not as good as hoped


Review: deal healthiest salt use box last family year problem iodized sea salt raise blood pressure regular salt 
Original summary: great deal 
Predicted summary:  great salt


Review: perfect size 